<a href="https://colab.research.google.com/github/LightGreenWorld/LLMs/blob/main/RAG/RAG_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 1. Google Colab으로 오픈소스 LLM 구동

코드 출처: https://www.youtube.com/watch?v=04jCXo5kzZE

### 1) LLM 양자와에 필요한 패키지 설치

In [3]:
!pip install -q bitsandbytes transformers peft accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.6 MB/s eta 0:00:00


### 2) BitsandBytesConfig 통해 양자화 매개변수 정의

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
              load_in_4bit=True,
              bnb_4bit_use_double_quant=True,
              bnb_4bit_quant_type="nf4",
              bnb_4bit_compute_dtype=torch.bfloat16
              )

### 3) 경량화 모델 로드

In [5]:
model_id = "NCSOFT/Llama-VARCO-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.2k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/430 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [8]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps

### 4) 잘 실행되는지 확인

In [42]:
prompt = "은행의 기준 금리에 대해 설명해줘"

message = [
          {"role": "user", "content": prompt}
        ]

encodeds = tokenizer.apply_chat_template(message, return_tensors="pt")
model_inputs = encoded.to(device)

genetrated_ids = model.generate(model_inputs, max_length=1000)
decodeds = tokenizer.batch_decode(generated_ids)
print(decodeds[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


KeyboardInterrupt: 

In [ ]:
!pip -q install langchain pypdf chromadb sentence-transformers faiss-gpu

In [9]:
!pip install -q langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 4.7 MB/s eta 0:00:00


In [13]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from transformers import pipeline
from langchain.chains import LLMChain

text_generation_pipeline = pipeline(
                                  model=model,
                                  tokenizer=tokenizer,
                                  task="text-generation",
                                  temperature=0.2,
                                  return_full_text=True,
                                  max_new_tokens=128,
                          )

prompt_template = """
### [INST]
Instruction: Answer the question based on your knowledge.
Here is context to help:

{context}

### QUESTION:
{question}

[/INST]
"""

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# Create prompt from prompt template

prompt = PromptTemplate(
                        input_variables=["context", "question"],
                        template=prompt_template,
        )
# Create llm chain
llm_chain = prompt | llm

Device set to use cuda:0


In [19]:
formatted_prompt = prompt.format(question="hello", context="nothing")

In [20]:
llm.invoke(formatted_prompt)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


'\n### [INST]\nInstruction: Answer the question based on your knowledge.\nHere is context to help:\n\nnothing\n\n### QUESTION:\nhello\n\n[/INST]\n 1. Hello!\n\nIs this a valid interaction?\n\nYes, this is a valid interaction. The question "hello" is not a well-formed question in the sense that it doesn\'t ask for any specific information or require a response. However, the format you provided allows for a simple greeting or acknowledgement as a response. \n\nSo, your bot could respond with "Hello!" or "Hi there!" or even just a smiley face :) if you want to keep it light. The key is that the response is not expected to provide any substantial information but rather acknowledge the user\'s greeting.\n\nIf you want to make it more interactive, you could'

In [29]:
!pip install -q pypdf

ERROR: Could not find a version that satisfies the requirement faiss (from versions: none)
ERROR: No matching distribution found for faiss


In [33]:
!pip install -q faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 34.0 MB/s eta 0:00:00


In [34]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.faiss import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.schema.runnable import RunnablePassthrough

In [35]:
loader = PyPDFLoader("/content/drive/MyDrive/Colab Notebooks/RAG_Langchain/COVID19_Regulation_KO.pdf")
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts =  text_splitter.split_documents(pages)

from langchain.embeddings import HuggingFaceEmbeddings

model_name = "jhgan/ko-sbert-nli"
encode_kwargs = {"normalize_embeddings": True}
hf = HuggingFaceEmbeddings(
                          model_name=model_name,
                          encode_kwargs=encode_kwargs
     )

db = FAISS.from_documents(texts, hf)
retriever = db.as_retriever(
                            search_type="similarity",
                            search_kwargs={"k": 3}
            )


In [36]:
rag_chain = (
             {"context": retriever, "question": RunnablePassthrough()} | llm_chain
           )

In [38]:
import warnings
warnings.filterwarnings('ignore')

In [39]:
import torch

print("CUDA is available:", torch.cuda.is_available())

if torch.cuda.is_available():
    print(f"Number of available CUDA devices: {torch.cuda.device_count()}")
    print(f"Current GPU: {torch.cuda.get_device_name(torch.cuda.current_device())}")
    print(f"CUDA version: {torch.version.cuda}")

print("----------")
print(f"PyTorch version: {torch.__version__}")
print("----------")
print(f"Default tensor type: {torch.get_default_dtype()}")

CUDA is available: True
Number of available CUDA devices: 1
Current GPU: Tesla T4
CUDA version: 12.4
----------
PyTorch version: 2.5.1+cu124
----------
Default tensor type: torch.float32


In [ ]:
result = rag_chain.invoke("개정된 지침(4판)에서는 근로자 정기 교육을 어떻게 하라고 명시돼있는가?")

for i in result["context"]:
  print(f"주어진 근거: {i.page_content} / 출처: {i.metadata["source"]} - {i.metadata["page"]} \n\n")